In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import gc

In [2]:
def reduce_mem_usage(df, verbose=True):
    """
    faster
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2 
    dfs = []
    for col in df.columns: 
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    dfs.append(df[col].astype(np.int8))
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    dfs.append(df[col].astype(np.int16))
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    dfs.append(df[col].astype(np.int32))
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    dfs.append(df[col].astype(np.int64) ) 
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    dfs.append(df[col].astype(np.float16))
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    dfs.append(df[col].astype(np.float32))
                else:
                    dfs.append(df[col].astype(np.float64))
        else:
            dfs.append(df[col])
    
    df_out = pd.concat(dfs, axis=1)
    if verbose:
        end_mem = df_out.memory_usage().sum() / 1024**2
        num_reduction = str(100 * (start_mem - end_mem) / start_mem)
        print(f'Mem. usage decreased to {str(end_mem)[:3]}Mb:  {num_reduction[:2]}% reduction')
    return df_out

In [3]:
def reduce_mem_usage2(df, verbose=True):
    """
    base
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2 
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64) 

    if verbose:
        end_mem = df.memory_usage().sum() / 1024**2
        num_reduction = str(100 * (start_mem - end_mem) / start_mem)
        print(f'Mem. usage decreased to {str(end_mem)[:3]}Mb:  {num_reduction[:2]}% reduction')
    return df

In [4]:
def make_df(n_cols=10, n_rows=1000):
    data = [np.arange(n_rows) for i in range(n_cols)]
    df_out = pd.DataFrame(data,
                          dtype=np.int64)
    return df_out

In [5]:
def measure_exp_time(func, df_input):
    start = time.perf_counter()
    func(df_input)
    run_time = time.perf_counter() - start
    return run_time


def exp_run(ncols_list, n_rows, reduce_func, transpose=False):
    t = []
    for n_cols in ncols_list:
        df = make_df(n_cols=n_cols, n_rows=n_rows)
        if transpose:
            df = df.T
            
        t.append(
            measure_exp_time(reduce_func, df)
        )
        gc.collect()
    
    return t

In [6]:
def print_result(ncols_list, n_rows, transpose):
    exp1 = exp_run(ncols_list=ncols_list, 
                   n_rows=n_rows,
                   reduce_func=reduce_mem_usage,
                   transpose=False)


    exp2 = exp_run(ncols_list=ncols_list, 
                   n_rows=n_rows,
                   reduce_func=reduce_mem_usage2,
                   transpose=False)



    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(ncols_list, exp1, label="concat[1]")
    ax.plot(ncols_list, exp2, label="overwrite[2]")
    
    if transpose:
        ax.set_title(f"ncols : {n_rows}")
        ax.set_xlabel("n_rows")
    else:
        ax.set_title(f"nrows : {n_rows}")
        ax.set_xlabel("n_cols")
    ax.set_ylabel("time [s]")
    ax.set_xticks(ncols_list)

    ax.legend()
    plt.show()

Mem. usage decreased to 0.1Mb:  75% reduction
Mem. usage decreased to 1.8Mb:  75% reduction
Mem. usage decreased to 3.7Mb:  75% reduction
Mem. usage decreased to 0.1Mb:  75% reduction
Mem. usage decreased to 1.8Mb:  75% reduction
Mem. usage decreased to 3.7Mb:  75% reduction


In [7]:
n_rows = 10000
ncols_list = [10, 100, 200]
transpose=False
print_result(ncols_list, n_rows, transpose)

In [8]:
"""
n_rows = 10000
ncols_list = [10, 100, 200]
transpose=True
print_result(ncols_list, n_rows, transpose)
"""

'\nn_rows = 10000\nncols_list = [10, 100, 200]\ntranspose=True\nprint_result(ncols_list, n_rows, transpose)\n'